In [ ]:
from google.colab import files
uploaded = files.upload()


Saving survey_results_public.csv to survey_results_public.csv


In [ ]:
import pandas as pd

# Replace the name with the actual file name if different
df = pd.read_csv("survey_results_public.csv", low_memory=False)

# Now check shape
df.shape


(65437, 114)

In [ ]:
# Filter: only full-time employed developers with salary info
df = df[
    (df['Employment'] == 'Employed, full-time') &
    (df['ConvertedCompYearly'].notnull())
].copy()

# Check how much data is left
print("Filtered dataset shape:", df.shape)
df[['Employment', 'ConvertedCompYearly']].head()


Filtered dataset shape: (17586, 114)


,Employment,ConvertedCompYearly
374,"Employed, full-time",30074.0
379,"Employed, full-time",91295.0
392,"Employed, full-time",161044.0
395,"Employed, full-time",121018.0
398,"Employed, full-time",195000.0


In [ ]:
# Select relevant columns
df = df[['EdLevel', 'YearsCodePro', 'Country', 'ConvertedCompYearly']].copy()

# Preview
print("Selected dataset shape:", df.shape)
df.head()


Selected dataset shape: (17586, 4)


,EdLevel,YearsCodePro,Country,ConvertedCompYearly
374,"Professional degree (JD, MD, Ph.D, Ed.D, etc.)",6,Austria,30074.0
379,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",6,Turkey,91295.0
392,"Professional degree (JD, MD, Ph.D, Ed.D, etc.)",18,United Kingdom of Great Britain and Northern I...,161044.0
395,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",25,United Kingdom of Great Britain and Northern I...,121018.0
398,Some college/university study without earning ...,30,United States of America,195000.0


In [ ]:
# Step 4.1: Convert YearsCodePro to numeric (some values may be "Less than 1 year", etc.)
df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')

# Step 4.2: Drop rows with missing values (YearsCodePro, EdLevel, Country)
df.dropna(subset=['YearsCodePro', 'EdLevel', 'Country'], inplace=True)

# Step 4.3: Reset index
df.reset_index(drop=True, inplace=True)

# Step 4.4: Encode EdLevel and Country
from sklearn.preprocessing import LabelEncoder
le_edu = LabelEncoder()
le_country = LabelEncoder()

df['EdLevel'] = le_edu.fit_transform(df['EdLevel'])
df['Country'] = le_country.fit_transform(df['Country'])

# Preview
df.head()


,EdLevel,YearsCodePro,Country,ConvertedCompYearly
0,4,6.0,8,30074.0
1,2,6.0,140,91295.0
2,4,18.0,145,161044.0
3,2,25.0,145,121018.0
4,6,30.0,147,195000.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Define features and target
X = df.drop('ConvertedCompYearly', axis=1)
y = df['ConvertedCompYearly']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)

# Train models
lr.fit(X_train, y_train)
rf.fit(X_train, y_train)

# Predict
y_pred_lr = lr.predict(X_test)
y_pred_rf = rf.predict(X_test)

# Evaluation function
def evaluate_model(name, y_true, y_pred):
    print(f"\n📊 {name} Performance:")
    print("R² Score:", r2_score(y_true, y_pred))
    print("MAE     :", mean_absolute_error(y_true, y_pred))
    print("RMSE    :", np.sqrt(mean_squared_error(y_true, y_pred)))

# Evaluate both
evaluate_model("Linear Regression", y_test, y_pred_lr)
evaluate_model("Random Forest Regressor", y_test, y_pred_rf)



📊 Linear Regression Performance:
R² Score: 0.1615328717495388
MAE     : 45191.44502412875
RMSE    : 68963.7999094933

📊 Random Forest Regressor Performance:
R² Score: -0.46535500331905477
MAE     : 35485.72490483142
RMSE    : 91169.50323420975


In [ ]:
def predict_salary(education_level_text, years_experience, country_text):
    # Encode inputs
    ed_level_encoded = le_edu.transform([education_level_text])[0]
    country_encoded = le_country.transform([country_text])[0]

    # Format into DataFrame
    input_df = pd.DataFrame([[ed_level_encoded, years_experience, country_encoded]],
                            columns=["EdLevel", "YearsCodePro", "Country"])

    # Predict
    predicted_salary = lr.predict(input_df)[0]

    return f"💰 Predicted Salary: ${predicted_salary:,.2f} USD"


In [ ]:
predict_salary("Master’s degree (M.A., M.S., M.Eng., MBA, etc.)", 5, "United States of America")


'💰 Predicted Salary: $98,009.34 USD'

In [ ]:
import joblib

# Export the trained model and encoders
joblib.dump(lr, "model_lr.pkl")
joblib.dump(le_edu, "le_edu.pkl")
joblib.dump(le_country, "le_country.pkl")


['le_country.pkl']

In [ ]:
from google.colab import files

# Download all the saved files
files.download("model_lr.pkl")
files.download("le_edu.pkl")
files.download("le_country.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Load the uploaded CSV (adjust name if needed)
df = pd.read_csv("/survey_results_public.csv", low_memory=False)

# Preview columns
print("Number of columns:", len(df.columns))
print("Column names:")
print(df.columns.tolist())


Number of columns: 114
Column names:
['ResponseId', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'Check', 'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline', 'TechDoc', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'BuildvsBuy', 'TechEndorse', 'Country', 'Currency', 'CompTotal', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'DatabaseAdmired', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'WebframeAdmired', 'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 'ToolsTechAdmired', 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'NEWCollabToolsAdmired', 'OpSysPersonal use', 'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWit

In [ ]:
# Step 2: Preview 'LanguageHaveWorkedWith' values
df["LanguageHaveWorkedWith"].dropna().sample(10, random_state=42)


,LanguageHaveWorkedWith
28590,JavaScript;Python;TypeScript
15783,C#;JavaScript;SQL
7921,Bash/Shell (all shells);C;Clojure;HTML/CSS;Jav...
4898,Java;JavaScript;TypeScript
4747,C;C#;C++;Delphi;HTML/CSS;Java;JavaScript;PHP;P...
21427,Bash/Shell (all shells);Go;HTML/CSS;JavaScript...
17461,Go;HTML/CSS;JavaScript;SQL;TypeScript
13116,Groovy;HTML/CSS;Java;JavaScript;SQL
2980,Java;JavaScript;TypeScript
27295,C;C#;C++;HTML/CSS;JavaScript;PHP;SQL;TypeScript


In [ ]:
# Step 3: One-hot encode the languages
lang_dummies = df["LanguageHaveWorkedWith"].str.get_dummies(sep=';')

# Join these new columns to your main DataFrame
df_lang = pd.concat([df, lang_dummies], axis=1)

# Preview new structure
df_lang[lang_dummies.columns].head()


,Ada,Apex,Assembly,Bash/Shell (all shells),C,C#,C++,Clojure,Cobol,Crystal,...,Rust,SQL,Scala,Solidity,Swift,TypeScript,VBA,Visual Basic (.Net),Zephyr,Zig
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
# Count how many developers use each language
top_langs = lang_dummies.sum().sort_values(ascending=False).head(10)

# Show the top 10 languages
print("Top 10 languages:\n", top_langs)

# Keep only these columns
df_lang_filtered = pd.concat([df, lang_dummies[top_langs.index]], axis=1)

# Preview
df_lang_filtered[top_langs.index].head()


Top 10 languages:
 JavaScript                 18715
HTML/CSS                   15843
SQL                        15357
Python                     14920
TypeScript                 11716
Bash/Shell (all shells)    10129
Java                        8796
C#                          8214
C++                         6574
C                           5764
dtype: int64


,JavaScript,HTML/CSS,SQL,Python,TypeScript,Bash/Shell (all shells),Java,C#,C++,C
0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,1,1,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0
3,1,1,1,1,1,0,1,0,1,1
4,1,1,0,1,0,0,0,0,1,0


In [ ]:
# Select final input features
X = df_lang_filtered[["EdLevel", "YearsCodePro", "Country"] + list(top_langs.index)]

# Output/label
y = df_lang_filtered["ConvertedCompYearly"]

# Drop rows with missing values
final_data = pd.concat([X, y], axis=1).dropna()

# Final inputs and outputs
X_final = final_data.drop("ConvertedCompYearly", axis=1)
y_final = final_data["ConvertedCompYearly"]

# Confirm shape
print("✅ Final input shape:", X_final.shape)
print("✅ Final label shape:", y_final.shape)


✅ Final input shape: (17340, 13)
✅ Final label shape: (17340,)


In [ ]:
# Convert 'YearsCodePro' to numeric
X_encoded["YearsCodePro"] = (
    X_encoded["YearsCodePro"]
    .replace("Less than 1 year", "0")
    .replace("More than 50 years", "51")
)

# Then convert to float
X_encoded["YearsCodePro"] = X_encoded["YearsCodePro"].astype(float)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_final, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("📊 Model Performance with Language Features:")
print(f"R² Score      : {r2}")
print(f"MAE           : {mae}")
print(f"RMSE          : {rmse}")


📊 Model Performance with Language Features:
R² Score      : 0.14291157585105108
MAE           : 46165.18404446614
RMSE          : 81223.10237026779


In [ ]:
import os

# Delete old files if they exist
for file in ["model_lr.pkl", "le_edu.pkl", "le_country.pkl"]:
    if os.path.exists(file):
        os.remove(file)
        print(f"✅ Deleted old {file}")
    else:
        print(f"❌ {file} not found (no need to delete)")


✅ Deleted old model_lr.pkl
✅ Deleted old le_edu.pkl
✅ Deleted old le_country.pkl


In [ ]:
# ✅ Import everything needed
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ✅ Delete old files if present
for file in ["model_lr.pkl", "le_edu.pkl", "le_country.pkl"]:
    if os.path.exists(file):
        os.remove(file)
        print(f"✅ Deleted old {file}")

# ✅ Recreate encoders
le_edu = LabelEncoder()
le_country = LabelEncoder()

# Use df_lang_filtered and X_encoded from earlier steps
X_encoded = X_final.copy()

# ✅ Fix YearsCodePro
X_encoded["YearsCodePro"] = (
    X_encoded["YearsCodePro"]
    .replace("Less than 1 year", "0")
    .replace("More than 50 years", "51")
).astype(float)

# ✅ Encode EdLevel
le_edu.fit(X_encoded["EdLevel"].unique())
X_encoded["EdLevel"] = le_edu.transform(X_encoded["EdLevel"])

# ✅ Encode Country
le_country.fit(X_encoded["Country"].unique())
X_encoded["Country"] = le_country.transform(X_encoded["Country"])

# ✅ Split data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_final, test_size=0.2, random_state=42)

# ✅ Train model
lr = LinearRegression()
lr.fit(X_train, y_train)

# ✅ Evaluate
y_pred = lr.predict(X_test)
print("\n📊 Model Performance with Language Features:")
print(f"R² Score      : {r2_score(y_test, y_pred)}")
print(f"MAE           : {mean_absolute_error(y_test, y_pred)}")
print(f"RMSE          : {np.sqrt(mean_squared_error(y_test, y_pred))}")

# ✅ Save model and encoders
joblib.dump(lr, "model_lr.pkl")
joblib.dump(le_edu, "le_edu.pkl")
joblib.dump(le_country, "le_country.pkl")



📊 Model Performance with Language Features:
R² Score      : 0.14291157585105108
MAE           : 46165.18404446614
RMSE          : 81223.10237026779


['le_country.pkl']

In [ ]:
from google.colab import files

files.download("model_lr.pkl")
files.download("le_edu.pkl")
files.download("le_country.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

df = pd.read_csv("/survey_results_public.csv", low_memory=False)

# Print total number of columns and preview all names
print(f"🧾 Total columns: {len(df.columns)}\n")
print("📋 Column names:")
print(df.columns.tolist())

# Show a few rows
df.head(2)


🧾 Total columns: 114

📋 Column names:
['ResponseId', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'Check', 'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline', 'TechDoc', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'BuildvsBuy', 'TechEndorse', 'Country', 'Currency', 'CompTotal', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'DatabaseAdmired', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'WebframeAdmired', 'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 'ToolsTechAdmired', 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'NEWCollabToolsAdmired', 'OpSysPersonal use', 'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWi

,ResponseId,MainBranch,Age,Employment,RemoteWork,Check,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,Under 18 years old,"Employed, full-time",Remote,Apples,Hobby,Primary/elementary school,Books / Physical media,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [ ]:
columns = [
    "EdLevel", "YearsCodePro", "Country", "RemoteWork",
    "DevType", "CodingActivities", "LanguageHaveWorkedWith", "ConvertedCompYearly"
]

df_clean = df[columns].copy()

# Drop rows with missing salary or experience
df_clean = df_clean.dropna(subset=["ConvertedCompYearly", "YearsCodePro", "EdLevel", "Country"])

# Reset index
df_clean.reset_index(drop=True, inplace=True)

print("✅ Cleaned data shape:", df_clean.shape)
df_clean.head()


✅ Cleaned data shape: (23345, 8)


,EdLevel,YearsCodePro,Country,RemoteWork,DevType,CodingActivities,LanguageHaveWorkedWith,ConvertedCompYearly
0,"Secondary school (e.g. American high school, G...",1,Pakistan,"Hybrid (some remote, some in-person)",Data scientist or machine learning specialist,Hobby;School or academic work;Professional dev...,Assembly;Bash/Shell (all shells);C;C++;HTML/CS...,7322.0
1,"Professional degree (JD, MD, Ph.D, Ed.D, etc.)",6,Austria,"Hybrid (some remote, some in-person)",Academic researcher,Hobby;School or academic work;Professional dev...,C;C++;Fortran;MATLAB;Python;SQL,30074.0
2,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",6,Turkey,Remote,Data scientist or machine learning specialist,Hobby;Bootstrapping a business,JavaScript;Julia;Python;SQL;TypeScript,91295.0
3,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",17,France,Remote,"Developer, back-end",Hobby,Python,53703.0
4,Some college/university study without earning ...,7,United States of America,Remote,Student,Hobby;School or academic work,HTML/CSS;JavaScript;Python;SQL;TypeScript,110000.0


In [ ]:
import numpy as np

# Convert 'YearsCodePro' to numeric (handle 'Less than 1 year' etc.)
df_clean["YearsCodePro"] = df_clean["YearsCodePro"].replace("Less than 1 year", "0")
df_clean["YearsCodePro"] = df_clean["YearsCodePro"].replace("More than 50 years", "51")
df_clean["YearsCodePro"] = df_clean["YearsCodePro"].astype(float)

# Binary feature: HasCertification → if 'Take certifications' is in CodingActivities
df_clean["HasCertification"] = df_clean["CodingActivities"].apply(
    lambda x: 1 if isinstance(x, str) and "certification" in x.lower() else 0
)

# Encode RemoteWork (one-hot: Remote, In-person, Hybrid)
df_remote = pd.get_dummies(df_clean["RemoteWork"], prefix="Remote")

# Encode DevType (multi-label one-hot encoding)
df_devtype = df_clean["DevType"].str.get_dummies(sep=";")

# Encode Languages (top 10)
top_languages = [
    'JavaScript', 'HTML/CSS', 'SQL', 'Python', 'TypeScript',
    'Bash/Shell (all shells)', 'Java', 'C#', 'C++', 'C'
]
df_lang = df_clean["LanguageHaveWorkedWith"].str.get_dummies(sep=';')
df_lang = df_lang[top_languages].copy()

# Label encode Education and Country
from sklearn.preprocessing import LabelEncoder
le_edu = LabelEncoder()
df_clean["EdLevel_encoded"] = le_edu.fit_transform(df_clean["EdLevel"])

le_country = LabelEncoder()
df_clean["Country_encoded"] = le_country.fit_transform(df_clean["Country"])

# Final feature matrix
X = pd.concat([
    df_clean[["EdLevel_encoded", "YearsCodePro", "Country_encoded", "HasCertification"]],
    df_remote.reset_index(drop=True),
    df_devtype.reset_index(drop=True),
    df_lang.reset_index(drop=True)
], axis=1)

# Target
y = df_clean["ConvertedCompYearly"]

# Output shape
print("✅ Final input shape:", X.shape)
print("✅ Final label shape:", y.shape)


✅ Final input shape: (23345, 51)
✅ Final label shape: (23345,)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict and evaluate
y_pred = lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("\n📊 Final Model Performance:")
print(f"R² Score : {r2}")
print(f"MAE      : {mae}")
print(f"RMSE     : {rmse}")

# Save model and encoders
joblib.dump(lr, "model_lr.pkl")
joblib.dump(le_edu, "le_edu.pkl")
joblib.dump(le_country, "le_country.pkl")



📊 Final Model Performance:
R² Score : 0.10358919320788684
MAE      : 48404.761195159575
RMSE     : 112102.39300592488


['le_country.pkl']

In [ ]:
from google.colab import files

files.download("model_lr.pkl")
files.download("le_edu.pkl")
files.download("le_country.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
joblib.dump(list(X.columns), "model_features.pkl")


['model_features.pkl']

In [ ]:
files.download("model_lr.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import joblib
joblib.dump(list(X.columns), "model_features.pkl")


['model_features.pkl']

In [ ]:
from google.colab import files
files.download("model_features.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>